In [2]:
!pip3 install -q transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.6 MB/s 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
model_path = f"Afia-manubea/FineTuned-DistilBert-Model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model= f"Afia-manubea/FineTuned-DistilBert-Model")

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# Input preprocessing
text = "This covid came with its own agenda"
text = preprocess(text)

In [ ]:
# PyTorch-based models
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# TensorFlow-based models
# model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
# model.save_pretrained(model_path)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

In [ ]:
encoded_input

{'input_ids': tensor([[  101,  2023,  2522, 17258,  2234,  2007,  2049,  2219, 11376,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5647,  0.9987, -0.4073]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# score without softmax function

output[0][0].detach().numpy()

array([-0.5647182 ,  0.998708  , -0.40728953], dtype=float32)

In [ ]:
#display score with softmax function

scores

array([0.14397496, 0.6875027 , 0.16852231], dtype=float32)

In [ ]:
config.id2label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}

In [ ]:
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
print (f"Classified text:{text}")
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Classified text:This covid came with its own agenda
1) NEUTRAL 0.6875
2) POSITIVE 0.1685
3) NEGATIVE 0.144


# **GRADIO APP**

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")
model = AutoModelForSequenceClassification.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")

def sentiment_analysis(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass through the model
    with torch.no_grad():
        output = model(**inputs)

    # Extract the predicted probabilities
    scores = torch.nn.functional.softmax(output.logits, dim=1).squeeze().tolist()

    # Define the sentiment labels
    labels = ["Negative", "Neutral", "Positive"]

    # Create a dictionary of sentiment scores
    scores_dict = {label: score for label, score in zip(labels, scores)}

    return scores_dict

demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Write/Type your tweet here"),
    outputs="Tweet",
    #intrepretation="default",
    examples=[
        ["Vaccine Who!, and where"],
        ["There's a global pandemic ongoing called Covid"],
        ["Covid is dangerous"],
        ["Covid is affecting Businesses badly"],
        ["This so-called Covid is not going to block our shine. Come to The beach this weekend! It's going to be lit"],
    ],
    title="Covid Tweets Sentiment Analysis App",
    description="This Application is the interface to Our Sentiment Analysis Model fine-tuned from a DistilBERT model.",
)

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d660c4e02ceee91ee1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")
model = AutoModelForSequenceClassification.from_pretrained("Afia-manubea/FineTuned-DistilBert-Model")

def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model(**inputs)
    scores = torch.nn.functional.softmax(output.logits, dim=1).squeeze().tolist()
    labels = ["Negative", "Neutral", "Positive"]
    scores_dict = {label: score for label, score in zip(labels, scores)}
    return scores_dict

def home():
    return "Welcome to the Covid Tweets Sentiment Analysis App!"

def predict(tweet):
    scores = sentiment_analysis(tweet)
    return f"The sentiment of the tweet is: {max(scores, key=scores.get)}"

def about():
    return "This application is an interface to our Sentiment Analysis Model fine-tuned from a DistilBERT model."


iface = gr.Interface(
    fn=home,
    inputs="text",
    outputs="text",
    title="Covid Tweets Sentiment Analysis App",
    description="**Menu:**\n\n- [Home](this_fn='home')\n- [Predict](this_fn='predict')\n- [About](this_fn='about')",
    examples=[
        ["Vaccine Who!, and where"],
        ["There's a global pandemic ongoing called Covid"],
        ["Covid is dangerous"],
        ["Covid is affecting Businesses badly"],
        ["This so-called Covid is not going to block our shine. Come to The beach this weekend! It's going to be lit"],
    ],
)
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:819: UserWarning: Expected 0 arguments for function <function home at 0x7e3cabdb6e60>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:827: UserWarning: Expected maximum 0 arguments for function <function home at 0x7e3cabdb6e60>, received 1.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://040d83e88747e7cc52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
